# Chapter 3.1: RAG Fundamentals

In this notebook, you'll learn how to build **Retrieval-Augmented Generation (RAG)** systems from the ground up. We'll start by creating a search index programmatically, then build RAG manually to understand the mechanics before showing how function tools can give agents retrieval capabilities.

---

## Learning Objectives

By the end of this notebook, you will:

1. Create an Azure AI Search index programmatically using Python
2. Understand the RAG pattern and why it's essential for enterprise AI
3. Build a "raw" RAG implementation manually to feel the complexity
4. Wrap search as a function tool for agent-controlled retrieval

---

## Prerequisites

Before starting this notebook, complete the setup guide: `03.0-setup-guide.md`

You should have:
- An Azure AI Search service created
- A Storage account with blob container
- Proper permissions configured
- Environment variables in your `.env` file

In [ ]:
# Environment Setup & Validation
import sys
sys.path.insert(0, '..')

from workshop_utils import validate_env
validate_env()

---

## Part 1: Programmatic Index Setup

Instead of using the Azure Portal wizard, we'll create our search index using Python. This approach is:

- **Reproducible** - Run the same code in any environment
- **Version controlled** - Index definitions live with your code
- **CI/CD friendly** - Automate index creation in pipelines

### Step 1.1: Load Configuration

In [24]:
import os
from dotenv import load_dotenv

load_dotenv()

# Azure AI Search configuration
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("SEARCH_API_KEY")
index_name = 'test-index-2'

# Azure OpenAI configuration (same as other notebooks)
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")

print(f"Search Endpoint: {search_endpoint}")
print(f"Index Name: {index_name}")
print(f"OpenAI Endpoint: {openai_endpoint}")

Search Endpoint: https://maf-workshop-ai-search.search.windows.net
Index Name: test-index-2
OpenAI Endpoint: https://aoai-sweden-gbb-dev.openai.azure.com/


### Step 1.2: Define the Index Schema

An Azure AI Search index consists of:
- **Fields** - The data structure (like database columns)
- **Vector Search Configuration** - Settings for embedding-based search
- **Semantic Configuration** - Settings for AI-powered ranking

In [25]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticSearch,
    SemanticPrioritizedFields,
    SemanticField,
)
from azure.core.credentials import AzureKeyCredential

# Define the index fields (matching Azure AI Search portal wizard schema)
# Field names: chunk_id, parent_id, chunk, title, text_vector
fields = [
    SearchField(
        name="chunk_id",
        type=SearchFieldDataType.String,
        key=True,
        sortable=True,
        filterable=True,
    ),
    SearchField(
        name="parent_id",
        type=SearchFieldDataType.String,
        filterable=True,  # For filtering chunks by source document
    ),
    SearchField(
        name="chunk",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="title",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="text_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=3072,  # text-embedding-3-large dimension
        vector_search_profile_name=f"{index_name}-vector-profile",
    ),
]

# Configure vector search (matching portal wizard settings)
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name=f"{index_name}-algorithm",
            parameters=HnswParameters(
                metric="cosine",
                m=4,
                ef_construction=400,
                ef_search=500,
            ),
        ),
    ],
    profiles=[
        VectorSearchProfile(
            name=f"{index_name}-vector-profile",
            algorithm_configuration_name=f"{index_name}-algorithm",
        ),
    ],
)

# Configure semantic search (matching portal wizard)
semantic_config = SemanticConfiguration(
    name=f"{index_name}-semantic-configuration",
    prioritized_fields=SemanticPrioritizedFields(
        content_fields=[SemanticField(field_name="chunk")],
        title_field=SemanticField(field_name="title"),
    ),
)

semantic_search = SemanticSearch(
    default_configuration_name=f"{index_name}-semantic-configuration",
    configurations=[semantic_config],
)

# Create the index definition
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search,
)

print(f"Index schema defined with {len(fields)} fields")
print(f"Fields: {[f.name for f in fields]}")

Index schema defined with 5 fields
Fields: ['chunk_id', 'parent_id', 'chunk', 'title', 'text_vector']


### Step 1.3: Create the Index

In [26]:
# Create the index client
index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=AzureKeyCredential(search_key),
)

# Create or update the index
try:
    result = index_client.create_or_update_index(index)
    print(f"✅ Index '{result.name}' created/updated successfully")
except Exception as e:
    print(f"❌ Error creating index: {e}")

✅ Index 'test-index-2' created/updated successfully


### Step 1.4: Load and Chunk Documents

We need to:
1. Read the PDF documents from `data/index1/`
2. Split them into chunks
3. Generate embeddings for each chunk
4. Upload to the index

In [27]:
import hashlib
from pathlib import Path
import fitz


def extract_text_from_pdf(pdf_path: str) -> str:
    """Extract text content from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text


def chunk_text(text: str, chunk_size: int = 2000, overlap: int = 500) -> list[str]:
    """
    Split text into overlapping chunks.
    
    Default settings match the Azure AI Search portal wizard:
    - maximumPageLength: 2000 characters
    - pageOverlapLength: 500 characters
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        if chunk.strip():  # Only add non-empty chunks
            chunks.append(chunk)
        start = end - overlap
    return chunks


# Load documents from data/index1/
data_path = Path("data/index2")
documents = []

for pdf_file in data_path.glob("*.pdf"):
    print(f"Processing: {pdf_file.name}")
    text = extract_text_from_pdf(str(pdf_file))
    chunks = chunk_text(text)
    
    # parent_id is the hash of the source document (for grouping chunks)
    parent_id = hashlib.md5(pdf_file.name.encode()).hexdigest()
    
    for i, chunk in enumerate(chunks):
        # chunk_id uniquely identifies each chunk (matches portal wizard format)
        chunk_id = hashlib.md5(f"{pdf_file.name}_{i}".encode()).hexdigest()
        documents.append({
            "chunk_id": chunk_id,
            "parent_id": parent_id,
            "chunk": chunk,
            "title": pdf_file.name,
        })

print(f"\nTotal chunks to index: {len(documents)}")

Processing: PerksPlus.pdf
Processing: role_library.pdf
Processing: Benefit_Options.pdf
Processing: employee_handbook.pdf

Total chunks to index: 55


### Step 1.5: Generate Embeddings

We'll use Azure OpenAI to generate vector embeddings for each chunk.

In [28]:
from openai import AzureOpenAI

# Setup Azure OpenAI client for embeddings (using API key like other notebooks)
openai_client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=openai_endpoint,
    api_key=openai_api_key,
)


def get_embedding(text: str) -> list[float]:
    """Generate embedding for a text chunk."""
    response = openai_client.embeddings.create(
        input=text,
        model=embedding_deployment
    )
    return response.data[0].embedding


# Generate embeddings for all chunks (this may take a few minutes)
print("Generating embeddings...")
for i, doc in enumerate(documents):
    doc["text_vector"] = get_embedding(doc["chunk"])  # Field name matches portal wizard
    if (i + 1) % 10 == 0:
        print(f"  Processed {i + 1}/{len(documents)} chunks")

print(f"✅ Embeddings generated for {len(documents)} chunks")

Generating embeddings...
  Processed 10/55 chunks
  Processed 20/55 chunks
  Processed 30/55 chunks
  Processed 40/55 chunks
  Processed 50/55 chunks
✅ Embeddings generated for 55 chunks


### Step 1.6: Upload Documents to Index

In [29]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Upload documents to the index
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key)
)

# Upload in batches (Azure Search has a limit of 1000 documents per batch)
batch_size = 100
for i in range(0, len(documents), batch_size):
    batch = documents[i:i + batch_size]
    result = search_client.upload_documents(documents=batch)
    succeeded = sum(1 for r in result if r.succeeded)
    print(f"Uploaded batch {i // batch_size + 1}: {succeeded}/{len(batch)} succeeded")

print(f"\n✅ Document upload complete!")

Uploaded batch 1: 55/55 succeeded

✅ Document upload complete!


---

## Part 2: Understanding RAG

### What is Retrieval-Augmented Generation?

**RAG** is a pattern that enhances LLM responses by retrieving relevant information from external knowledge sources before generating an answer.

```
┌─────────────────────────────────────────────────────────────────┐
│                        RAG Architecture                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   User Query ──────┬──────────────────────────────────────────► │
│                    │                                            │
│                    ▼                                            │
│         ┌─────────────────────┐                                 │
│         │   Azure AI Search   │  ◄── Vectorized Documents       │
│         │   (Retrieval)       │                                 │
│         └─────────┬───────────┘                                 │
│                   │                                             │
│                   │ Retrieved Context                           │
│                   ▼                                             │
│         ┌─────────────────────┐                                 │
│         │   LLM (Generation)  │  ◄── Query + Context            │
│         └─────────┬───────────┘                                 │
│                   │                                             │
│                   ▼                                             │
│            Grounded Response                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Why RAG?

| Problem | How RAG Solves It |
|---------|-------------------|
| **Hallucination** | Grounds responses in retrieved facts |
| **Stale Knowledge** | Retrieves from up-to-date indexes |
| **Context Limits** | Only retrieves relevant chunks, not entire documents |
| **Data Privacy** | Keeps proprietary data in your own indexes |

---

## Part 3: Manual RAG - Feel the Pain

Before using abstractions, let's understand what's happening under the hood. We'll build a RAG system step-by-step:

1. Connect to Azure AI Search
2. Create a search function
3. Manually inject retrieved context into the prompt
4. Generate a response

This "raw" approach gives you full control and helps you understand the mechanics.

### Step 3.1: Perform a Raw Search

Let's see what the search returns **before** we add any LLM processing. This is the "Retrieval" part of RAG.

In [17]:
from azure.search.documents.models import VectorizedQuery


def search_documents(query: str, top_k: int = 3) -> list[dict]:
    """Search the index using hybrid search (text + vector)."""
    # Generate embedding for the query
    query_vector = get_embedding(query)
    
    # Create vector query targeting the text_vector field (matches portal wizard)
    vector_query = VectorizedQuery(
        vector=query_vector,
        k=top_k,
        fields="text_vector"  # Field name matches portal wizard schema
    )
    
    # Perform hybrid search
    results = search_client.search(
        search_text=query,  # Text search on 'chunk' field
        vector_queries=[vector_query],
        select=["chunk_id", "parent_id", "chunk", "title"],
        top=top_k
    )
    
    return [{
        "chunk_id": r["chunk_id"],
        "title": r["title"],
        "chunk": r["chunk"][:200] + "...",
        "score": r["@search.score"]
    } for r in results]


# Test the search
test_results = search_documents("What mental health services are covered?")
for r in test_results:
    print(f"📄 {r['title']} (score: {r['score']:.3f})")
    print(f"   {r['chunk']}")
    print()

📄 Northwind_Health_Plus_Benefits_Details.pdf (score: 0.025)
   . This will help ensure that your transportation request is processed in 
time for your appointment. Additionally, be sure to have your Northwind Health Plus 
member ID card with you when you receive ...

📄 Northwind_Health_Plus_Benefits_Details.pdf (score: 0.024)
   orthwind Health Plus offers a 
network of providers that are in-network, including primary care physicians, specialists, 
hospitals, and pharmacies.  
Out-of-Network Services 
In some cases, it may be...

📄 Northwind_Health_Plus_Benefits_Details.pdf (score: 0.024)
   nity and newborn care, preventive and 
wellness services, mental health and substance abuse services, and prescription drugs. The 
plan also must provide coverage for preventive services without cost ...



### Step 3.2: Manual RAG - Injecting Context into the Prompt

Now let's manually construct a prompt that includes the retrieved context. This is the core RAG pattern - **retrieve, then generate**.

In [20]:
from agent_framework.azure import AzureOpenAIChatClient

# Create chat client using API key (same pattern as other notebooks)
chat_client = AzureOpenAIChatClient(
    endpoint=openai_endpoint,
    api_key=openai_api_key,
    api_version=api_version,
    deployment_name=chat_deployment,
)


async def manual_rag(user_question: str) -> str:
    """
    Perform RAG manually:
    1. Search for relevant documents
    2. Build a prompt with the context
    3. Generate a response
    """
    # Step 1: Retrieve relevant context
    retrieved_docs = search_documents(user_question, top_k=3)
    
    # Step 2: Format context for the prompt
    context_text = "\n\n".join([
        f"[Source: {doc['title']}]\n{doc['chunk']}" 
        for doc in retrieved_docs
    ])
    
    # Step 3: Build the augmented prompt
    system_prompt = f"""You are an HR assistant for Contoso Electronics.
Answer questions based ONLY on the following retrieved context.
If the context doesn't contain the answer, say "I don't have that information."

## Retrieved Context:
{context_text}
"""
    
    # Step 4: Generate response using the agent
    agent = chat_client.as_agent(
        name="manual_rag_agent",
        instructions=system_prompt
    )
    
    response = await agent.run(user_question)
    return response.text


# Test manual RAG
question = "What is the deductible for Northwind Health Plus?"
answer = await manual_rag(question)

print(f"Question: {question}\n")
print(f"Answer: {answer}")

Question: What is the deductible for Northwind Health Plus?

Answer: The Northwind Health Plus plan has a calendar year deductible that applies to some services. However, I don't have the specific deductible amount.


### The Problems with Manual RAG

While this works, there are several issues:

| Problem | Description |
|---------|-------------|
| **Boilerplate Code** | You must write retrieval logic for every agent |
| **No Multi-turn Memory** | Context isn't automatically managed across conversation turns |
| **Token Management** | You must manually handle context window limits |
| **Multiple Sources** | Combining multiple indexes requires custom orchestration |
| **Serialization** | Persisting conversation state requires custom code |

In the next section, we'll see how Function Tools can improve this, and in the next notebook, how **Context Providers** solve all these problems elegantly.

---

## Part 4: Function Tools for Search

An intermediate approach is to wrap the search as a **function tool** that the agent can call. This gives the agent **agency** - it decides when to search, rather than searching on every query.

### When to Use Tools vs. Context Providers

| Approach | Use When |
|----------|----------|
| **Function Tool** | Agent should decide when to search (optional retrieval) |
| **Context Provider** | Every query needs grounding (automatic retrieval) |

In [21]:
from typing import Annotated
import json
from azure.core.exceptions import HttpResponseError


class AzureAISearchTool:
    """
    A function tool that wraps Azure AI Search for use by an agent.
    
    The agent decides when to call this tool based on the user's question.
    """
    
    def __init__(self, search_client: SearchClient, index_name: str):
        self.search_client = search_client
        self.index_name = index_name
    
    def search(
        self, 
        query: Annotated[str, "The search query to find relevant documents"],
        top: Annotated[int, "Number of results to return"] = 5
    ) -> str:
        """
        Search the Contoso HR knowledge base for information about health plans,
        policies, and employee benefits.
        
        Use this tool to find:
        - Health plan details (Northwind Health Plus, Northwind Standard)
        - Coverage information, deductibles, and copays
        - Eligibility requirements and enrollment procedures
        - Company HR policies and benefits
        """
        try:
            results = self.search_client.search(
                search_text=query,
                top=top,
                query_type="semantic",
                semantic_configuration_name=f"{self.index_name}-semantic-configuration",
            )
            
            docs = []
            for doc in results:
                docs.append({
                    "content": doc.get("chunk", ""),
                    "score": doc.get("@search.score", 0),
                    "source": doc.get("title", "Unknown"),
                })
            
            return json.dumps(docs, indent=2)
            
        except HttpResponseError as e:
            return json.dumps({"error": str(e)})

In [22]:
# Create the search tool
search_tool = AzureAISearchTool(search_client=search_client, index_name=index_name)

# Create an agent with the search tool
tool_agent = chat_client.as_agent(
    name="hr_assistant",
    instructions="""
    You are an HR Assistant for Contoso Electronics specializing in employee health plans.
    
    IMPORTANT RULES:
    1. ALWAYS use the search tool before answering questions about health plans
    2. Base your responses ONLY on information from the search results
    3. If the search returns no relevant information, say so explicitly
    4. Cite which plan (Northwind Health Plus or Standard) the information comes from
    """,
    tools=[search_tool.search]
)

# Test the tool-based agent
response = await tool_agent.run("What are the out-of-network copays?")
print(response.text)

Here is what I found about out-of-network copays for each health plan:

**Northwind Health Plus:**
- Out-of-network copays are typically higher than for in-network services.
- The actual copay amount for out-of-network primary care visits is not specified, but documentation states they are "higher than for in-network services."
- For out-of-network providers, you may sometimes be responsible for the entire cost of care, unless exceptions apply (such as certain emergency or international situations).
- You will also pay a deductible ($2,000/year) and 20% coinsurance for network services, but out-of-network coinsurance and copays are generally higher.
(Source: Northwind Health Plus Benefits Details)

**Northwind Standard:**
- Out-of-network costs are significantly higher than in-network.
- For out-of-network services, coinsurance is typically 40%.
- Copays for out-of-network services are not specified, but you may need to pay the full cost or a far greater portion than in-network.
- Prim

In [23]:
# Test with a question that spans multiple topics
response = await tool_agent.run(
    "Compare the deductibles between Northwind Health Plus and Northwind Standard"
)
print(response.text)

Based on the search results, here is a comparison of deductibles for both plans:

- **Northwind Health Plus**:
  - The calendar year deductible for in-network services is $1,500 for individuals and $3,000 for families. (Northwind Health Plus)
  - Certain services, such as preventive care and emergency services, are exempt from the deductible.

- **Northwind Standard**:
  - The calendar year deductible is $2,000 per person and $4,000 for each family. (Northwind Standard)
  - Preventive care services are covered at 100% with no deductible.
  - Prescription drugs have a separate deductible of $250 per individual and $500 per family.

**Summary**: Northwind Health Plus has a lower deductible for both individuals and families compared to Northwind Standard. Additionally, both plans waive the deductible for preventive care, but Northwind Standard also has a separate prescription drug deductible.

If you need further details on coinsurance or copays, let me know!


---

## Exercise 1: Improve the Search Tool

The current search tool returns raw JSON. Improve it by:

1. Formatting the results in a more readable way
2. Adding error handling for empty results
3. Limiting the content length per result to avoid token limits

<details>
<summary>Click to see solution</summary>

```python
def search(
    self, 
    query: Annotated[str, "The search query"],
    top: Annotated[int, "Number of results"] = 3
) -> str:
    """Search the knowledge base."""
    try:
        results = self.search_client.search(
            search_text=query,
            top=top,
            query_type="semantic"
        )
        
        documents = list(results)
        
        if not documents:
            return "No relevant documents found for this query."
        
        # Format results with truncated content
        formatted = []
        for i, doc in enumerate(documents, 1):
            content = doc.get("chunk", "")[:500]  # Limit to 500 chars
            source = doc.get("title", "Unknown")
            formatted.append(f"[{i}] {source}\n{content}")
        
        return "\n\n".join(formatted)
        
    except HttpResponseError as e:
        return f"Search error: {str(e)}"
```

</details>

---

## Summary

In this notebook, you learned:

1. **Programmatic Indexing** - How to create Azure AI Search indexes using Python code
2. **RAG Mechanics** - The retrieve-then-generate pattern and why it matters
3. **Manual RAG** - Building RAG from scratch to understand the complexity
4. **Function Tools** - Giving agents the ability to search when needed

### Key Takeaways

- Manual RAG works but requires significant boilerplate code
- Function tools give agents control over when to search
- Both approaches have limitations around multi-turn conversations and state management

### Next Steps

In **03.2 - Context Providers**, you'll learn how the Agent Framework's Context Provider abstraction solves all these problems elegantly:

- Automatic context injection on every invocation
- Built-in multi-turn conversation support
- Easy combination of multiple knowledge sources
- Clean, declarative configuration